### 交叉熵损失函数


#### **1. 二分类交叉熵（BCE, Binary Cross-Entropy）**
**公式**：
$$ L_{BCE} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right] $$

**参数说明**：
- $ N $：样本总数
- $ y_i $：第 i 个样本的真实标签（取值为 0 或 1）
- $ \hat{y}_i $：模型对第 i  个样本的预测概率（需经 Sigmoid 激活，范围 (0,1)）


#### **2. 多分类交叉熵（CCE, Categorical Cross-Entropy）**
**公式**：
$$ L_{CCE} = -\frac{1}{N} \sum_{i=1}^{N} \sum_{c=1}^{C} y_{i,c} \log(\hat{y}_{i,c}) $$

**参数说明**：
- $ N $：样本总数
- $ C $：类别总数
- $ y_{i,c} $：第 i 个样本在第 c 类的真实标签（One-Hot 编码，1 表示属于该类，0 表示不属于）
- $ \hat{y}_{i,c} $：模型预测第 i 个样本属于第 c 的概率（需经 Softmax 激活，满足 $ \sum_{c=1}^{C} \hat{y}_{i,c} = 1 $）


#### **核心区别**
| **场景**| 二分类交叉熵| 多分类交叉熵|
|----------------|----------------------------|----------------------------|
| **标签类型**| 单个二值标签（0/1）| One-Hot 向量（类别互斥）|
| **激活函数**| Sigmoid（输出单个概率）| Softmax（输出概率分布）|
| **求和维度**| 无类别维度（单标签）| 需对类别维度求和（多标签） |

## 手撕交叉熵

In [13]:
import torch
import numpy as np


def manual_softmax(logits):
    # 防止数值溢出，减去最大值
    max_vals = torch.max(logits, dim=-1, keepdim=True).values
    exp_logits = torch.exp(logits - max_vals)
    return exp_logits / torch.sum(exp_logits, dim=-1, keepdim=True)

def manual_cross_entropy(logits, labels):
    # 计算 softmax 概率
    probs = manual_softmax(logits)

    # 数值稳定性处理：防止log(0)
    epsilon = 1e-7
    probs = torch.clamp(probs, min=epsilon, max=1.0 - epsilon)
    
    # 将标签转为 one-hot 编码
    one_hot = torch.zeros_like(probs)
    one_hot[torch.arange(len(labels)), labels] = 1.0
    # 计算交叉熵: -sum(p * log(q))
    log_probs = torch.log(probs)
    cross_entropy = -torch.sum(one_hot * log_probs) / len(labels)
    return cross_entropy

In [14]:
import torch
import numpy as np


# --- 使用示例 ---
# 假设一个简单的多分类模型（例如，一个线性层，输出3个值）
class SimpleMultiClassNN(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.linear = torch.nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x) # 输出原始logits，不在这里做Softmax

# 创建模型（输入特征2维，输出3类）
model = SimpleMultiClassNN(input_size=2, num_classes=3)

# 示例数据：2个样本，每个样本2个特征
features = torch.tensor([[0.1, 0.2],
                         [0.7, 0.8]], dtype=torch.float32)
# 真实标签：第一个样本属于第0类，第二个样本属于第2类
labels = torch.tensor([0, 2], dtype=torch.long) # 注意 dtype 是 long

# 前向传播，得到原始logits
logits = model(features)
print("Model raw logits:\n", logits)

# 计算损失
loss = manual_cross_entropy(logits, labels)
print("Multiclass Cross-Entropy Loss:", loss.item())

# 实际上，PyTorch提供了更高效且数值稳定的内置函数：
# nn.CrossEntropyLoss (它已经将Softmax和交叉熵计算合并优化)
criterion = torch.nn.CrossEntropyLoss()
loss_pytorch = criterion(logits, labels)
print("PyTorch CrossEntropyLoss:", loss_pytorch.item())

Model raw logits:
 tensor([[-0.0532,  0.6314,  0.4303],
        [-0.1757,  1.1589,  0.5889]], grad_fn=<AddmmBackward0>)
Multiclass Cross-Entropy Loss: 1.3504061698913574
PyTorch CrossEntropyLoss: 1.3504061698913574
